In [8]:
import pyodbc

# Verbindingsgegevens
server = '127.0.0.1'        
port = '1433'               
database = 'GreatOutdoors_SDM'         
username = 'SA'             
password = 'iDTyjZx7dRL4'  

# Connection string
connection_string = (
    f"DRIVER={{ODBC Driver 17 for SQL Server}};"
    f"SERVER={server},{port};"
    f"DATABASE={database};"
    f"UID={username};"
    f"PWD={password};"
    "TrustServerCertificate=yes;"
    "Timeout=30;"
)

# Maak verbinding met de database
conn = pyodbc.connect(connection_string)
cursor = conn.cursor()

# Zet FOREIGN KEY constraints tijdelijk uit
cursor.execute("EXEC sp_MSforeachtable 'ALTER TABLE ? NOCHECK CONSTRAINT ALL'")
conn.commit()

# Haal alle tabellen op in de database
cursor.execute("SELECT TABLE_NAME FROM INFORMATION_SCHEMA.TABLES WHERE TABLE_TYPE = 'BASE TABLE'")
tables = [row.TABLE_NAME for row in cursor.fetchall()]

# Loop door alle tabellen en verwijder de data
for table in tables:
    try:
        cursor.execute(f"DELETE FROM {table}")  # Verwijder alle rijen
        
        # Check of de tabel een IDENTITY-kolom heeft
        cursor.execute(f"""
            SELECT 1 
            FROM sys.columns 
            WHERE object_id = OBJECT_ID('{table}') 
            AND columnproperty(object_id, name, 'IsIdentity') = 1
        """)
        has_identity = cursor.fetchone()

        # Reset alleen als er een identity-kolom is
        if has_identity:
            cursor.execute(f"DBCC CHECKIDENT ('{table}', RESEED, 0)")
        
        conn.commit()
        print(f"✅ Tabel {table} geleegd {'en IDENTITY gereset' if has_identity else '(geen IDENTITY)'}")
    except pyodbc.Error as e:
        print(f"❌ Fout bij {table}: {e}")

# Zet FOREIGN KEY constraints weer aan
cursor.execute("EXEC sp_MSforeachtable 'ALTER TABLE ? CHECK CONSTRAINT ALL'")
conn.commit()

# Sluit de verbinding
cursor.close()
conn.close()

print("🎉 Database volledig gereset!")


✅ Tabel age_group geleegd (geen IDENTITY)
✅ Tabel sales_territory geleegd (geen IDENTITY)
✅ Tabel country geleegd (geen IDENTITY)
✅ Tabel retailer_segment geleegd (geen IDENTITY)
✅ Tabel retailer_type geleegd (geen IDENTITY)
✅ Tabel retailer_headquarters geleegd (geen IDENTITY)
✅ Tabel retailer geleegd (geen IDENTITY)
✅ Tabel retailer_site geleegd (geen IDENTITY)
✅ Tabel retailer_contact geleegd (geen IDENTITY)
✅ Tabel sales_demographic geleegd (geen IDENTITY)
✅ Tabel order_method geleegd (geen IDENTITY)
✅ Tabel product_line geleegd (geen IDENTITY)
✅ Tabel return_reason geleegd (geen IDENTITY)
✅ Tabel sales_branch geleegd (geen IDENTITY)
✅ Tabel product_type geleegd (geen IDENTITY)
✅ Tabel product geleegd (geen IDENTITY)
✅ Tabel sales_staff geleegd (geen IDENTITY)
✅ Tabel order_header geleegd (geen IDENTITY)
✅ Tabel order_details geleegd (geen IDENTITY)
✅ Tabel returned_item geleegd (geen IDENTITY)
✅ Tabel course geleegd (geen IDENTITY)
✅ Tabel satisfaction_type geleegd (geen IDENTITY)